<a href="https://colab.research.google.com/github/hacker-yorik/Face-Recognition-/blob/master/Copy_of_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import PIL

width=120
height=120

from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
data=tf.data.Dataset.list_files("/content/gdrive/My Drive/lfw-deepfunneled/*/*")




In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
def give_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == "Georg_Maier"

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [width, height])

def process_path(file_path):
    print(type(file_path))
    title=give_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img,title


In [ ]:
def speed_up_data():
  data_speed=tf.data.Dataset.list_files("/content/gdrive/My Drive/lfw-deepfunneled/Aaron_Eckhart/Aaron_Eckhart_0001.jpg")
  return data_speed.map(process_path).batch(1)

data_to_speed=speed_up_data()

<class 'tensorflow.python.framework.ops.Tensor'>


In [6]:
labeled_ds = data.map(process_path)
type(labeled_ds)
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE=1

labeled_ds=labeled_ds.batch(BATCH_SIZE)

#tensored=tf.convert_to_tensor(labeled_ds)
# labeled_ds=tf.reshape(labeled_ds,(1,120,120,3))

print((labeled_ds))


<class 'tensorflow.python.framework.ops.Tensor'>
<BatchDataset shapes: ((None, 120, 120, 3), (None,)), types: (tf.float32, tf.bool)>


In [26]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(4, (2,2), strides=(1,1), activation="relu", input_shape=(120, 120,3)),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(4, (2,2), strides=(1,1), activation="relu"),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(4, (2,2), strides=(1,1), activation="relu"),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(4, (2,2), strides=(1,1), activation="relu"),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Conv2D(4, (2,2), strides=(1,1), activation="relu"),
#     tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

network_input = tf.keras.Input(shape=(120, 120, 3), name="img")

#First conv layer
x1 = tf.keras.layers.Conv2D(60, (4,4), strides=(1,1), activation="relu")(network_input)
x1 = tf.keras.layers.MaxPooling2D((2,2))(x1)

for i in range(8):
    #First skip layer
    x2 = tf.keras.layers.Conv2D(60, (2,2), strides=(1,1), activation="relu", padding="same")(x1)
    x2 = tf.keras.layers.Conv2D(60, (2,2), strides=(1,1), activation=None, padding="same")(x2)
    #Skip connection
    x3 = tf.keras.layers.Add()([x1,x2])
    x3 = tf.keras.layers.ReLU()(x3)
    x1 = x3

network_output = tf.keras.layers.AvgPool2D((4,4))(x3)
network_output = tf.keras.layers.Flatten()(network_output)
network_output = tf.keras.layers.Dense(1,activation="sigmoid")(network_output)
model = tf.keras.Model(network_input, network_output, name="MyResNet")

In [27]:
model.summary()

Model: "MyResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 120, 120, 3) 0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 117, 117, 60) 2940        img[0][0]                        
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 58, 58, 60)   0           conv2d_42[0][0]                  
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 58, 58, 60)   14460       max_pooling2d_8[0][0]            
___________________________________________________________________________________________

In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
def fit_model(input_data,epochs):
  with tf.device('/device:GPU:0'):
    model.fit(input_data, epochs=epochs)


# fit_model(data_to_speed,2)

fit_model(labeled_ds,10)

Epoch 1/10
14096/14096 [==============================] - 261s 18ms/step - loss: 0.4311 - accuracy: 0.9378
Epoch 2/10
 9688/14096 [===================>..........] - ETA: 1:20 - loss: 0.1173 - accuracy: 0.9683

In [22]:
# model.save("/content/gdrive/My Drive/Model_5_ResNet_epochs=20")
# model=tf.keras.models.load_model("/content/gdrive/My Drive/Model_3_epochs=20")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Model_5_ResNet_epochs=20/assets


In [23]:
import os
paths=os.listdir("/content/gdrive/My Drive/Test-FaceRecognition")
total_score=0
total_pics=0
for path in paths:
  files=os.listdir("/content/gdrive/My Drive/Test-FaceRecognition/"+path)
  total_pics+=len(files)

  for pic in files:
    #load pic and transform into predictable form
    test_im=PIL.Image.open("/content/gdrive/My Drive/Test-FaceRecognition/"+path+"/"+pic)
    test_im=np.asarray(test_im,dtype="int32")
    test_im=tf.convert_to_tensor(test_im,dtype=tf.float32)
    test_im=tf.reshape(test_im,(1,120,120,3))

    score=model.predict(test_im)[0][0]
    if (path!='Georg' and score < 0.5) or (path=='Georg' and score >= 0.5):
      total_score+=1
print(total_score/total_pics)

0.5284810126582279


In [ ]:
score_myface=0
name=""
pic_=len(os.listdir("/content/gdrive/My Drive/Test-FaceRecognition/"+name))
for i in range(pic_):
    test_im=PIL.Image.open("/content/gdrive/My Drive/Test-FaceRecognition/"+name+"/"+name+"%04d" %i+ ".jpg")
    test_im=np.asarray(test_im,dtype="int32")
    test_im=tf.convert_to_tensor(test_im,dtype=tf.float32)
    test_im=tf.reshape(test_im,(1,120,120,3))
    if model.predict(test_im)[0][0] >=0.5:
      score_myface+=1
print(score_myface,pic_,score_myface/pic_)

114 142 0.8028169014084507
